In [1]:
import glob
import os.path

import matplotlib.pyplot as plt
import mne.io
import numpy as np
import yasa

mne.set_log_level('CRITICAL')

In [2]:
path_data = r'C:\Users\HP\PycharmProjects\as_monkey_eeg\X8_data'
animals = os.listdir(path_data)


In [3]:
def acc_to_locomotion(acc):
    loco = np.std(np.concatenate([np.zeros((3, 1)), np.diff(acc)], axis=1), axis=0).reshape(1, -1)
    return loco

for animal in animals:
    sessions = os.listdir(os.path.join(path_data, animal))
    
    for session in sessions:
    
        path_session = os.path.join(path_data, animal, session)
        
        file_eeg = glob.glob(os.path.join(path_session, '**eeg.edf'), recursive=True)[0]
        file_acc = glob.glob(os.path.join(path_session, '**acc.edf'), recursive=True)[0]
        
        raw = mne.io.read_raw_edf(file_eeg, preload=True)
        raw2 = mne.io.read_raw_edf(file_acc, preload=True)
        
        raw.pick('EEG0')
        
        loco = acc_to_locomotion(raw2.get_data()) / 1e6
        info = mne.create_info(ch_names=['Loco'], sfreq=raw2.info['sfreq'], ch_types='bio', verbose=False)
        raw3 = mne.io.RawArray(loco, info, verbose=False)
        raw3.resample(raw.info['sfreq'])
        
        tmax = min([raw.times[-1], raw3.times[-1]])
        raw.crop(0,tmax)
        raw3.crop(0,tmax)
        
        data = np.concatenate([raw.get_data(), raw3.get_data()], axis=0)
        info = mne.create_info(ch_names=['EEG', 'Loco'], sfreq=raw.info['sfreq'], ch_types=['eeg', 'bio'], verbose=False)
        raw = mne.io.RawArray(data, info, verbose=False)
        raw.resample(250)
        raw.export(os.path.join(path_session, 'raw.edf'), overwrite=True)